In [1]:
from constants import JQL, FIELDS, BASE_URL, JIRA_DOMAIN,EMAIL, MAX_RESULTS, MODULE_DEVS, VALID_STATUSES, MAIL_MAP, DAILY_HOURS,MIN_PROJECT_RATIO,PROJECT_MAP, DEFAULT_END_DATE, DEFAULT_END_DATE_with_timezone, DEFAULT_START_DATE,DEFAULT_START_DATE_with_timezone
from token_hidden import API_TOKEN



In [2]:
import os
import json
import time
import csv
from typing import List, Dict, Any, Set

import requests
from requests.auth import HTTPBasicAuth

# ========= Config =========


AUTH = HTTPBasicAuth(EMAIL, API_TOKEN)
HEADERS = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}


# JQL principal (ajustar a lo que uses)
JQL = """
project = IT
AND issuetype NOT IN (Epic, Sub-task, Subtarea)
ORDER BY priority DESC, duedate ASC
""".strip()

# Campos que querés en los issues "base"
MAIN_FIELDS = [
    "summary",
    "project",
    "reporter",
    "assignee",
    "status",
    "priority",
    "issuetype",
    "timetracking",
    "duedate",
    "customfield_10016",  # Story Points
    "customfield_10212",  # Módulo (ejemplo)
    "customfield_10214",
    "customfield_10442",
    "customfield_10608",
    "issuelinks"          # ¡necesario para ver vínculos!
]

# Campos que querés traer del issue vinculado (además de summary/status/priority, etc.)
WANTED_FIELDS = [
    "customfield_10209",
]

MAX_RESULTS = 100
CHUNK_LINKED = 50  # tamaño de lote para buscar issues vinculados
OUT_JSON = "jira_it_issues.json"
  # opcional
JIRA_API_ROOT = "https://team-1583163151751.atlassian.net/rest/api/3"
SEARCH_JQL_URL = f"{JIRA_API_ROOT}/search/jql"  

# ========= Funciones utilitarias =========
def fetch_issues_paged(jql: str, fields: list[str]) -> list[dict]:
    all_issues = []
    next_token = None
    while True:
        payload = {
            "jql": jql,
            "fields": fields,          # lista (incluí siempre 'issuelinks')
            "maxResults": MAX_RESULTS, # mismo nombre
        }
        if next_token:
            payload["nextPageToken"] = next_token

        resp = requests.post(SEARCH_JQL_URL, headers=HEADERS, auth=AUTH, json=payload, timeout=30)
        resp.raise_for_status()
        data = resp.json()

        issues = data.get("issues", [])
        all_issues.extend(issues)

        # Nueva paginación
        if data.get("isLast", False):
            break
        next_token = data.get("nextPageToken")
        if not next_token:  # fallback defensivo
            break
        time.sleep(0.2)

    return all_issues


def collect_linked_issue_keys(issues: List[Dict[str, Any]], only_types: List[str] = None) -> Set[str]:
    """
    Recolecta todas las keys de inward/outward de issuelinks.
    only_types: si lo pasás, filtrás por nombre de tipo de vínculo (p.ej. ["Problem/Incident"])
    """
    keys: Set[str] = set()
    for it in issues:
        links = it.get("fields", {}).get("issuelinks", []) or []
        for link in links:
            if only_types and link.get("type", {}).get("name") not in only_types:
                continue
            for side in ("outwardIssue", "inwardIssue"):
                if side in link and "key" in link[side]:
                    keys.add(link[side]["key"])
    print(f"🔗 Linked keys found: {len(keys)}")
    return keys


def fetch_issues_by_keys(keys: list[str], fields: list[str], chunk: int = CHUNK_LINKED) -> dict[str, dict]:
    out = {}
    for i in range(0, len(keys), chunk):
        slice_keys = keys[i:i+chunk]
        jql = f"issuekey in ({','.join(slice_keys)})"

        next_token = None
        while True:
            payload = {
                "jql": jql,
                "fields": fields,
                "maxResults": 100,
            }
            if next_token:
                payload["nextPageToken"] = next_token

            resp = requests.post(SEARCH_JQL_URL, headers=HEADERS, auth=AUTH, json=payload, timeout=30)
            resp.raise_for_status()
            data = resp.json()

            for issue in data.get("issues", []):
                out[issue["key"]] = issue

            if data.get("isLast", False):
                break
            next_token = data.get("nextPageToken")
            if not next_token:
                break
            time.sleep(0.2)
    return out

def enrich_issue_links_with_fields(issues: List[Dict[str, Any]],
                                   linked_map: Dict[str, Dict[str, Any]],
                                   fields_to_copy: List[str]) -> None:
    """Inyecta 'fields_to_copy' dentro de outward/inwardIssue.fields si NO existen aún en el issue base."""
    for it in issues:
        links = it.get("fields", {}).get("issuelinks", []) or []
        for link in links:
            for side in ("outwardIssue", "inwardIssue"):
                if side in link and "key" in link[side]:
                    k = link[side]["key"]
                    if k in linked_map:
                        link[side].setdefault("fields", {})
                        src_fields = linked_map[k].get("fields", {})
                        for f in fields_to_copy:
                            # ⛔️ Solo insertar si NO estaba definido antes
                            if f not in link[side]["fields"]:
                                link[side]["fields"][f] = src_fields.get(f)

def check_links_integrity(before: List[Dict[str, Any]], after: List[Dict[str, Any]]) -> None:
    """Verifica que los vínculos críticos como 'Blocks' no se hayan perdido tras enriquecer."""
    for b_issue, a_issue in zip(before, after):
        b_links = b_issue.get("fields", {}).get("issuelinks", [])
        a_links = a_issue.get("fields", {}).get("issuelinks", [])
        b_blocks = {(l.get("type", {}).get("name"), l.get("outwardIssue", {}).get("key"), l.get("inwardIssue", {}).get("key")) for l in b_links if l.get("type", {}).get("name") == "Blocks"}
        a_blocks = {(l.get("type", {}).get("name"), l.get("outwardIssue", {}).get("key"), l.get("inwardIssue", {}).get("key")) for l in a_links if l.get("type", {}).get("name") == "Blocks"}

        if b_blocks != a_blocks:
            print(f"🚨 WARNING: Issue {b_issue['key']} had Blocks links altered!")

def flatten_links_to_csv(issues: List[Dict[str, Any]],
                         csv_path: str,
                         sides: List[str] = ("outwardIssue", "inwardIssue"),
                         fields_for_flat: List[str] = None) -> None:
    """
    Aplana vínculos a CSV: una fila por (issue base, vínculo).
    fields_for_flat: columnas a sacar del vinculado (además de key y type).
    """
    if fields_for_flat is None:
        fields_for_flat = ["summary", "status", "priority", "duedate", "customfield_10016"]

    rows = []
    for it in issues:
        base_key = it.get("key")
        base_summary = it.get("fields", {}).get("summary")
        links = it.get("fields", {}).get("issuelinks", []) or []
        for link in links:
            link_type = link.get("type", {}).get("name")
            for side in sides:
                linked = link.get(side)
                if not linked:
                    continue
                lkey = linked.get("key")
                lf = (linked.get("fields") or {})
                row = {
                    "base_issue": base_key,
                    "base_summary": base_summary,
                    "link_type": link_type or "",
                    "link_side": side,
                    "linked_issue": lkey,
                }
                # Agregar columnas pedidas
                # status/priority vienen como objetos; sacar el "name" si existe
                for col in fields_for_flat:
                    val = lf.get(col)
                    if isinstance(val, dict) and "name" in val:
                        val = val["name"]
                    row[col] = val
                rows.append(row)

    # Escribir CSV
    fieldnames = ["base_issue", "base_summary", "link_type", "link_side", "linked_issue"] + fields_for_flat
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)
    print(f"📄 Links CSV saved: {csv_path} ({len(rows)} rows)")


# ========= Main =========
if __name__ == "__main__":
    # 1) Descargar issues base (con issuelinks)
    base_issues = fetch_issues_paged(JQL, MAIN_FIELDS)

    # 2) Recolectar keys de issues vinculados
    linked_keys = collect_linked_issue_keys(base_issues, only_types=None)  # podés filtrar por tipo si querés

    # 3) Traer info de issues vinculados (campos personalizados deseados)
    linked_map = {}
    if linked_keys:
        linked_map = fetch_issues_by_keys(
            list(linked_keys),
            fields=WANTED_FIELDS,
        )

    # 4) Enriquecer los vínculos en la estructura original
    base_issues_before = base_issues
    if linked_map:
        enrich_issue_links_with_fields(base_issues, linked_map, WANTED_FIELDS)

    check_links_integrity(base_issues_before, base_issues)
    # 5) Guardar JSON enriquecido
    with open(OUT_JSON, "w", encoding="utf-8") as f:
        json.dump({"issues": base_issues}, f, indent=2, ensure_ascii=False)
    print(f"💾 JSON enriched saved: {OUT_JSON}")




### Download epics
def fetch_epics():
    all_epics = {}

    # Epics del proyecto
    jql_epics = 'project = IT AND issuetype = Epic'
    for_epics = {"jql": jql_epics, "fields": ["key","duedate","summary"], "maxResults": 100}

    next_token = None
    while True:
        payload = dict(for_epics, **({"nextPageToken": next_token} if next_token else {}))
        r = requests.post(SEARCH_JQL_URL, headers=HEADERS, auth=AUTH, json=payload, timeout=30)
        r.raise_for_status()
        data = r.json()

        for issue in data.get("issues", []):
            epic_key = issue["key"]
            f = issue.get("fields", {}) or {}
            all_epics[epic_key] = {
                "due_date": f.get("duedate"),
                "summary": f.get("summary", ""),
                "tasks": []
            }

            # Historias de esa épica
            jql_stories = f'"Epic Link" = {epic_key}'
            next_story = None
            while True:
                story_payload = {
                    "jql": jql_stories,
                    "fields": ["key","customfield_10016","status"],
                    "maxResults": 100
                }
                if next_story:
                    story_payload["nextPageToken"] = next_story

                sr = requests.post(SEARCH_JQL_URL, headers=HEADERS, auth=AUTH, json=story_payload, timeout=30)
                sr.raise_for_status()
                sd = sr.json()

                for st in sd.get("issues", []):
                    sf = st.get("fields", {}) or {}
                    all_epics[epic_key]["tasks"].append({
                        "key": st["key"],
                        "story_points": sf.get("customfield_10016"),
                        "status": (sf.get("status") or {}).get("name", "Sin estado")
                    })

                if sd.get("isLast", False):
                    break
                next_story = sd.get("nextPageToken")
                if not next_story:
                    break

        if data.get("isLast", False):
            break
        next_token = data.get("nextPageToken")
        if not next_token:
            break

    return all_epics

epics = fetch_epics()

# Guardar en archivo si lo deseas
with open("epics_due_lookup.json", "w") as f:
    json.dump(epics, f, indent=2)



🔗 Linked keys found: 5579
💾 JSON enriched saved: jira_it_issues.json


In [3]:
API_KEY = '346436483896-non6bmg405eh4avr5saql5m1r0r37rim.apps.googleusercontent.com'
client_secret = '346436483896-non6bmg405eh4avr5saql5m1r0r37rim.apps.googleusercontent.com'

In [4]:
import os
import datetime

from collections import defaultdict
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

# 📌 Permisos necesarios: lectura del calendario
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

# 📂 Archivos de autenticación
CREDENTIALS_PATH = 'client_secret.json'
TOKEN_PATH = 'token.json'



def obtener_credenciales():
    creds = None
    if os.path.exists(TOKEN_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_PATH, SCOPES)
            creds = flow.run_local_server(port=0)
        with open(TOKEN_PATH, 'w') as token:
            token.write(creds.to_json())
    return creds

def obtener_bloques_ocupados(email, creds, start_dt, end_dt):
    service = build('calendar', 'v3', credentials=creds)
    eventos_resultado = service.events().list(
        calendarId=email,
        timeMin=start_dt.isoformat(),
        timeMax=end_dt.isoformat(),
        singleEvents=True,
        orderBy="startTime"
    ).execute()

    eventos = eventos_resultado.get('items', [])
    bloques_ocupados = []

    for evento in eventos:
        start_str = evento['start'].get('dateTime')
        end_str = evento['end'].get('dateTime')
        if not start_str or not end_str:
            continue  # Omitir eventos de todo el día

        start_time = datetime.datetime.fromisoformat(start_str)
        end_time = datetime.datetime.fromisoformat(end_str)
        bloques_ocupados.append((start_time, end_time))

    return bloques_ocupados

def obtener_bloques_por_dev():
    creds = obtener_credenciales()
    bloques_por_dev = {}

    for dev, email in MAIL_MAP.items():
        print(f"🔎 Consultando reuniones de {dev} ({email})...")
        try:
            bloques = obtener_bloques_ocupados(email, creds, DEFAULT_START_DATE_with_timezone, DEFAULT_END_DATE_with_timezone)
            bloques_por_dev[dev] = bloques
            #print(f"   ✅ {len(bloques)} reuniones encontradas.\n")
        except Exception as e:
            print(f"   ❌ Error consultando {email}: {str(e)}\n")
            bloques_por_dev[dev] = []

    return bloques_por_dev

# Para usar de forma aislada:
if __name__ == "__main__":
    bloques = obtener_bloques_por_dev()
    for dev, bloques_list in bloques.items():
        print(f"📅 {dev}: {len(bloques_list)} bloque(s) ocupado(s)")
        for start, end in bloques_list:
            print(f"   🕓 {start.strftime('%Y-%m-%d %H:%M')} → {end.strftime('%H:%M')}")
        

🔎 Consultando reuniones de Luis Uran (luisuran@biamex.com)...
🔎 Consultando reuniones de Diego Martin Gogorza (diegogogorza@biamex.com)...
🔎 Consultando reuniones de Nicolas Pardo (nicolaspardo@biamex.com)...
🔎 Consultando reuniones de Martín Horn (martinhorn@biamex.com)...
🔎 Consultando reuniones de Facundo Capua (facundocapua@biamex.com)...
🔎 Consultando reuniones de Franco Lorenzo (francolorenzo@biamex.com)...
🔎 Consultando reuniones de Alan Mori - Carestino (alanmori@biamex.com)...
🔎 Consultando reuniones de Gastón Ojeda (gastonojeda@biamex.com)...
🔎 Consultando reuniones de Miguel Armentano (miguelarmentano@biamex.com)...
🔎 Consultando reuniones de Juan Ignacio Morelis - Carestino (juanmorelis@biamex.com)...
🔎 Consultando reuniones de ivanalberghini (ivanalberghini@biamex.com)...
📅 Luis Uran: 10 bloque(s) ocupado(s)
   🕓 2025-12-22 08:30 → 09:00
   🕓 2025-12-23 08:30 → 09:00
   🕓 2025-12-24 08:30 → 09:00
   🕓 2025-12-24 09:00 → 09:30
   🕓 2025-12-25 08:30 → 09:00
   🕓 2025-12-26 0

In [5]:
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict


# === LOAD DATA ===
with open("jira_it_issues.json", "r", encoding="utf-8") as f:
    issues = json.load(f)["issues"]

with open("epics_due_lookup.json", "r", encoding="utf-8") as f:
    epic_due_lookup = json.load(f)

scheduled = []
tasks_by_dev = {}
issue_map = {}
graph = nx.DiGraph()

from datetime import timedelta


def _opt_value(v):
    """Si es opción (dict), devuelve .value/.name; si es lista, lista de values; otro -> tal cual."""
    if isinstance(v, dict):
        return v.get("value") or v.get("name")
    if isinstance(v, list):
        out = []
        for x in v:
            out.append(x.get("value") or x.get("name") if isinstance(x, dict) else x)
        return out
    return v

def get_cf10209_from_sd_outward(f_fields, only_link_type=None):
    """
    Busca en f_fields['issuelinks'] un outwardIssue cuya key empiece con 'SD-'
    y devuelve customfield_10209 (normalizado). Si only_link_type se setea,
    filtra por el nombre del tipo de vínculo (p.ej. 'Problem/Incident').
    """
    links = f_fields.get("issuelinks") or []
    for link in links:
        if only_link_type and link.get("type", {}).get("name") != only_link_type:
            continue
        out = link.get("outwardIssue")
        if not out:
            continue
        if not str(out.get("key", "")).startswith("SD-"):
            continue
        lfields = (out.get("fields") or {})
        val = _opt_value(lfields.get("customfield_10209"))
        if val:
            return val
    return None

def get_epic_key(fields):
    """
    Devuelve la key de la épica asociada al issue (si existe), probando:
    1) customfield_10008 (Epic Link) – clásico
    2) parent.key si el parent es de tipo 'Epic' – team-managed
    3) fields['epic'] (algunas instancias Cloud)
    """
    # 1) Epic Link clásico
    epic_link = fields.get("customfield_10008")
    if isinstance(epic_link, str) and epic_link:
        return epic_link

    # 2) parent -> Epic
    p = fields.get("parent")
    if isinstance(p, dict):
        p_issuetype = (p.get("fields") or {}).get("issuetype") or p.get("issuetype") or {}
        if str(p_issuetype.get("name", "")).lower() == "epic":
            return p.get("key")

    # 3) objeto 'epic'
    e = fields.get("epic")
    if isinstance(e, dict):
        return e.get("key") or e.get("id")

    return None

def contar_dias_laborables(start_date, end_date):
    dias = 0
    current = start_date
    while current <= end_date:
        if current.weekday() < 5:  # Lunes a Viernes
            dias += 1
        current += timedelta(days=1)
    return dias

for issue in issues:
    f = issue["fields"]
    status = f.get("status", {}).get("name", "")
    assignee = f.get("assignee")
    if not assignee:
        continue
    dev = assignee["displayName"]
    report = f.get("reporter")
    if not report:
        continue
    reporter = report["displayName"]
    key = issue["key"]
    summary = f.get("summary", "")
    module_info = f.get("customfield_10212")
    module_value = module_info["value"] if module_info else None
    estimate_hours = f.get("customfield_10608")

    if estimate_hours is None:
        estimate_hours = f.get("customfield_10016")

    if estimate_hours is None:
        estimate_seconds = f.get("timetracking", {}).get("originalEstimateSeconds") or f.get("aggregatetimeoriginalestimate")
        estimate_hours = estimate_seconds / 3600 if estimate_seconds else 0
    
    issue_to_epic = {}
    for ekey, edata in epic_due_lookup.items():
        for t in edata.get("tasks", []):
            k = t.get("key")
            if k:
                issue_to_epic[k] = ekey
    
    epic_key = get_epic_key(f) or issue_to_epic.get(key)  # 👈 fallback desde lookup
    epic_obj = epic_due_lookup.get(epic_key) if epic_key else None
    epic_due_str = (epic_obj or {}).get("due_date")

    # Prefiere due del issue; si no hay, usa el de la épica
    due_str = f.get("duedate") or epic_due_str
    try:
        due_date = datetime.strptime(due_str, "%Y-%m-%d") if due_str else datetime(2100, 1, 1)
    except:
        due_date = datetime(2100, 1, 1)

    epic_name = (epic_obj or {}).get("summary", "— Sin épica —")

    cf10442_raw = f.get("customfield_10442") or []
    cf10442_names = [u.get("displayName") for u in cf10442_raw if u.get("displayName")]
    suggested_devs = MODULE_DEVS.get(module_value, []) if module_value else []
    suggested_users = list(set(suggested_devs + cf10442_names))

    if dev in suggested_users:
        suggested_users.remove(dev)

    can_be_delegated = dev not in suggested_users and bool(suggested_users)
    delegation_note = f"🚨 Puede derivarse a: {', '.join(suggested_users)}" if can_be_delegated else ""



    cf10209_value = get_cf10209_from_sd_outward(
        f_fields=f,
        only_link_type="Problem/Incident"  # o None si no querés filtrar por tipo
    )
    
    task = {
        "key": key,
        "summary": summary,
        "estimate_hours": estimate_hours,
        "due_date": due_date,
        "assignee": dev,
        "status": status,
        "due_reason": None,
        "module": module_value,
        "suggested_users": suggested_users,
        "has_epic": bool(epic_key),
        "epic_name": epic_name, 
        "reporter": reporter,
        "cf10209": cf10209_value
    }
    # 🔧 Nuevo: guardar claves de tareas que bloquean esta
    
    issue_map[key] = task

    
    if status in VALID_STATUSES:
        tasks_by_dev.setdefault(dev, []).append(task)
    graph.add_node(key)
    

    # for key, task in issue_map.items():
    #     blocker_due_dates = [
    #         issue_map[b]["due_date"]
    #         for b in graph.predecessors(key)
    #         if b in issue_map and issue_map[b]["due_date"]
    #     ]

    #     if blocker_due_dates:
    #         max_blocker_due = min(blocker_due_dates)
    #         if task["due_date"] < max_blocker_due:
    #             task["due_date_original"] = task["due_date"]
    #             task["due_date"] = max_blocker_due

    for link in f.get("issuelinks", []):
        if "inwardIssue" in link and link["type"]["name"] == "Blocks":
            graph.add_edge(link["inwardIssue"]["key"], key)
        elif "outwardIssue" in link and link["type"]["name"] == "Blocks":
            graph.add_edge(key, link["outwardIssue"]["key"])

    blockers = [src for src, tgt in graph.edges() if tgt == key]
    task["blockers"] = blockers

for key, task in issue_map.items():
    # Para cada tarea, mirar a quién bloquea
    blocked_keys = list(graph.successors(key))
    
    blocked_due_dates = [
        issue_map[b]["due_date"]
        for b in blocked_keys
        if b in issue_map and issue_map[b]["due_date"]
    ]

    blocker_key = [issue_map[b]["key"] for b in blocked_keys if b in issue_map and issue_map[b]["key"]]

    if blocked_due_dates:
        min_blocked_due = min(blocked_due_dates)
        suggested_due = min_blocked_due - timedelta(days=1)
        task["note"] = (
            f"🕓 Fecha de vencimiento {'ajustada' if task['due_date'] > suggested_due else 'ya correcta'} "
            f"por bloqueo a tarjeta {blocker_key} que vence el: {suggested_due.strftime('%Y-%m-%d')}"
        )
        if task["due_date"] > suggested_due:
            task["due_date_original"] = task["due_date"]
            task["due_date"] = suggested_due
            task["note"] = f"🕓 Fecha de vencimiento ajustada por bloqueo a tarjeta {blocker_key} que vence el: {suggested_due.strftime('%Y-%m-%d')}"


# BLOQUES POR DEV (simulados)
bloques_por_dev = bloques
MINIMUM_SLOT_HOURS = 0.25
# === PLANIFICADOR POR DESARROLLADOR ===
def to_naive(dt):
    return dt.replace(tzinfo=None) if dt.tzinfo else dt

def planificar_tareas_para_dev(dev, tasks, bloques_ocupados):
    current_time = datetime.combine(DEFAULT_START_DATE.date(), datetime.strptime("08:30", "%H:%M").time())
    hours_per_day = DAILY_HOURS.get(dev, DAILY_HOURS["Default"])
    schedule = {}
    plan = []

    try:
        sorted_keys = list(nx.topological_sort(graph.subgraph([t["key"] for t in tasks])))
    except nx.NetworkXUnfeasible:
        sorted_keys = [t["key"] for t in tasks]
    sorted_keys.sort(key=lambda k: issue_map[k]["due_date"])

    def to_naive(dt):
        return dt.replace(tzinfo=None) if dt.tzinfo else dt
    
    for start, end in sorted(bloques_ocupados):
            plan.append({
                "developer": dev,
                "key": f"REUNION-{start.strftime('%Y%m%d%H%M')}",
                "summary": "⛔ Reunión",
                "has_epic": False,
                "due_date": start.date(),  # o end.date()
                "start": to_naive(start),
                "end": to_naive(end),
                "duration_hours": (to_naive(end) - to_naive(start)).total_seconds() / 3600,
                "type": "reunion"
            })

    for key in sorted_keys:
        task = issue_map[key]
        hours_left = task["estimate_hours"]
        start_time = None
       

        

        while hours_left > 0:
            date_str = current_time.strftime('%Y-%m-%d')
            used_today = schedule.get(date_str, 0)

            end_of_day = datetime.combine(current_time.date(), datetime.strptime("17:30", "%H:%M").time())
            available_time = (end_of_day - current_time).total_seconds() / 3600
            time_slot = min(available_time, hours_per_day - used_today, hours_left)

             # 🔧 NUEVO BLOQUE: limitar time_slot al hueco libre real antes del próximo conflicto
            next_conflict_start = None
            for start, end in sorted(bloques_ocupados):
                if to_naive(start) > to_naive(current_time):
                    next_conflict_start = to_naive(start)
                    break

            if next_conflict_start:
                gap_hours = (next_conflict_start - current_time).total_seconds() / 3600
                if gap_hours > 0:
                    time_slot = min(time_slot, gap_hours)

            if time_slot <= 0:
                next_day = current_time.date() + timedelta(days=1)
                while next_day.weekday() in [5, 6]:
                    next_day += timedelta(days=1)
                current_time = datetime.combine(next_day, datetime.strptime("08:30", "%H:%M").time())
                continue

            proposed_end = current_time + timedelta(hours=time_slot)

            bloque_conflictivo = any(
                to_naive(start) < to_naive(proposed_end) and to_naive(end) > to_naive(current_time)
                for start, end in bloques_ocupados
            )

            if bloque_conflictivo:
                conflictos = [
                    (start, end) for start, end in bloques_ocupados
                    if to_naive(start) < to_naive(proposed_end) and to_naive(end) > to_naive(current_time)
                ]
                conflictos.sort(key=lambda x: to_naive(x[0]))
                next_start = to_naive(conflictos[0][1])
                current_time = next_start
                continue

            if start_time is None:
                start_time = current_time

            if bloque_conflictivo:
                conflictos = [
                    (start, end) for start, end in bloques_ocupados
                    if to_naive(start) < to_naive(proposed_end) and to_naive(end) > to_naive(current_time)
                ]
                conflictos.sort(key=lambda x: to_naive(x[0]))
                next_start = to_naive(conflictos[0][1])
                current_time = next_start
                continue

            if start_time is None:
                start_time = current_time

            end_time = current_time + timedelta(hours=time_slot)
            plan.append({
                "developer": dev,
                "key": key,
                "summary": task["summary"],
                "has_epic": task["has_epic"],
                "epic_name": task["epic_name"],
                "due_date": task["due_date"],
                "start": current_time,
                "end": end_time,
                "duration_hours": time_slot,
                "suggested_users": task.get("suggested_users", []),
                "blockers": task.get("blockers", []),
                "note": task.get("note", ""),
                "reporter": task.get("reporter", ""),
                "cf10209": task.get("cf10209")
            })
            

            schedule[date_str] = used_today + time_slot
            hours_left -= time_slot
            current_time = end_time

    return plan

# === PLANIFICACIÓN INICIAL ===
# for dev in tasks_by_dev:
#     scheduled += planificar_tareas_para_dev(dev, tasks_by_dev[dev], bloques_por_dev[dev])

# === REBALANCEO ===
planned_hours_by_dev = defaultdict(float)
project_hours_by_dev = defaultdict(float)

for s in scheduled:
    if s.get("type") == "reunion":
        continue
    planned_hours_by_dev[s["developer"]] += s["duration_hours"]
    if issue_map[s["key"]]["has_epic"]:
        project_hours_by_dev[s["developer"]] += s["duration_hours"]

for dev in tasks_by_dev:
    all_tasks = tasks_by_dev[dev]
    
    within_range = [t for t in all_tasks if DEFAULT_START_DATE <= t["due_date"] <= DEFAULT_END_DATE]
    epic_tasks_in_range = sorted([t for t in within_range if t["has_epic"]], key=lambda t: t["due_date"])
    non_epic_tasks_in_range = sorted([t for t in within_range if not t["has_epic"]], key=lambda t: t["due_date"], reverse=True)
    epic_tasks_out_of_range = sorted(
        [t for t in all_tasks if t["has_epic"] and t not in within_range],
        key=lambda t: t["due_date"]
    )

    total_in_range = sum(t["estimate_hours"] for t in within_range)
    current_epic = sum(t["estimate_hours"] for t in epic_tasks_in_range)
    non_epic_hours = sum(t["estimate_hours"] for t in non_epic_tasks_in_range)
    min_ratio = MIN_PROJECT_RATIO.get(dev, MIN_PROJECT_RATIO["Default"])

    print(f"\n🔎 {dev}:")
    print(f"  🎯 Epic target: {min_ratio * 100:.1f}%")
    print(f"  📊 Total in range: {total_in_range:.1f} h | Epic in range: {current_epic:.1f} h")
    print(f"  🔍 Epic tasks out of range: {len(epic_tasks_out_of_range)}")
    for t in epic_tasks_out_of_range:
        print(f"    - {t['key']} ({t['estimate_hours']} h) due {t['due_date'].strftime('%Y-%m-%d')}")

    # 🟨 Forzar tareas con épica fuera de rango hasta cumplir el mínimo
    # horas_planificables_en_rango = (
    #         sum(t["estimate_hours"] for t in epic_tasks_in_range) +
    #         sum(t["estimate_hours"] for t in non_epic_tasks_in_range) +
    #         sum(t["estimate_hours"] for t in epic_tasks_out_of_range)
    #     )
    
    dias_laborables = contar_dias_laborables(DEFAULT_START_DATE, DEFAULT_END_DATE)
    horas_por_dia = DAILY_HOURS.get(dev, DAILY_HOURS["Default"])
    horas_planificables_en_rango = dias_laborables * horas_por_dia
    while epic_tasks_out_of_range:
        task = epic_tasks_out_of_range.pop(0)
        task_estimate = task["estimate_hours"]

        # Moverla dentro del rango
        task["due_date_original"] = task["due_date"]
        print(f" modificando {task['key']} due {t['due_date'].strftime('%Y-%m-%d')}")
        task["due_date"] = DEFAULT_START_DATE
        epic_tasks_in_range.append(task)
        current_epic += task_estimate

        # Recalcular ratio después de agregarla
        ratio_actual = current_epic / horas_planificables_en_rango if horas_planificables_en_rango > 0 else 0
        print(f"✔️ Ratio tras forzar {task['key']}: {current_epic:.1f} / ({horas_planificables_en_rango:.1f}) = {ratio_actual * 100:.1f}%")
        
        
        
        if ratio_actual >= min_ratio:
            break

    # ❌ Eliminar tareas sin épica si sigue sin cumplirse el mínimo
    ratio_actual = current_epic / (current_epic + non_epic_hours) if (current_epic + non_epic_hours) > 0 else 0
    while non_epic_tasks_in_range and ratio_actual < min_ratio:
        removed = non_epic_tasks_in_range.pop()
        non_epic_hours -= removed["estimate_hours"]
        ratio_actual = current_epic / (current_epic + non_epic_hours) if (current_epic + non_epic_hours) > 0 else 0

    # 🔄 Reordenar: primero épicas, luego tareas normales
    balanced_tasks = sorted(epic_tasks_in_range, key=lambda t: t["due_date"]) + \
                     sorted(non_epic_tasks_in_range, key=lambda t: t["due_date"])
    leftovers = [t for t in all_tasks if t not in balanced_tasks]
    tasks_by_dev[dev] = balanced_tasks + leftovers

    # 🔁 Replanificar solo ese desarrollador
    scheduled = [s for s in scheduled if s["developer"] != dev]
    scheduled += planificar_tareas_para_dev(dev, tasks_by_dev[dev], bloques_por_dev[dev])


# === RESUMEN FINAL ===
scheduled_by_dev_and_key = defaultdict(lambda: defaultdict(list))
for s in scheduled:
    scheduled_by_dev_and_key[s["developer"]][s["key"]].append(s)

print("\n📝 Developer Task Schedule Summary:\n")
for dev, tasks in tasks_by_dev.items():
    print(f"👨‍💻 Developer: {dev}")
    try:
        sorted_keys = list(nx.topological_sort(graph.subgraph([t["key"] for t in tasks])))
    except nx.NetworkXUnfeasible:
        sorted_keys = [t["key"] for t in tasks]
    sorted_keys.sort(key=lambda k: issue_map[k]["due_date"])
    for key in sorted_keys:
        task = issue_map[key]
        scheds = sorted(scheduled_by_dev_and_key[dev].get(key, []), key=lambda x: x["start"])
        if not scheds:
            continue
        start_time = scheds[0]["start"]
        end_time = scheds[-1]["end"]
        total_hours = sum(s["duration_hours"] for s in scheds)
        note_lines = []
        if "note" in task and task["note"]:
            note_lines.append(task["note"])

        # 🚨 Sugerencias de derivación
        if task.get("suggested_users"):
            note_lines.append(f"🚨 Puede derivarse a: {', '.join(task['suggested_users'])}")

        # 📛 Bloqueadores
        if task.get("blockers"):
            blocker_msgs = []
            for blocker_key in task["blockers"]:
                blocker = issue_map.get(blocker_key)
                if blocker:
                    blocker_msgs.append(f"{blocker_key}: {blocker['summary']} ({blocker['assignee']}, Due: {blocker['due_date'].strftime('%Y-%m-%d')})")
                else:
                    blocker_msgs.append(blocker_key)
            note_lines.append(f"📛 Debido a bloqueo de: " + " | ".join(blocker_msgs))

         # Agregar nota personalizada si existe
        if "note" in task and task["note"]:
            note_lines.append(task["note"])

        note = "\n     " + "\n     ".join(note_lines) if note_lines else ""
        print(
            f"   🔹 {key}: {round(total_hours, 1)}h → Due: {task['due_date'].strftime('%Y-%m-%d')} | "
            f"Finish: Start: {start_time.strftime('%Y-%m-%d %H:%M')} | End: {end_time.strftime('%Y-%m-%d %H:%M')}{note}"
        )
    print()


🔎 Nicolas Pardo:
  🎯 Epic target: 10.0%
  📊 Total in range: 11.0 h | Epic in range: 0.0 h
  🔍 Epic tasks out of range: 4
    - IT-21773 (4.0 h) due 2025-10-14
    - IT-22518 (16.0 h) due 2100-01-01
    - IT-22344 (16.0 h) due 2100-01-01
    - IT-22187 (12.0 h) due 2100-01-01
 modificando IT-21773 due 2100-01-01
✔️ Ratio tras forzar IT-21773: 4.0 / (80.0) = 5.0%
 modificando IT-22518 due 2100-01-01
✔️ Ratio tras forzar IT-22518: 20.0 / (80.0) = 25.0%

🔎 Miguel Armentano:
  🎯 Epic target: 50.0%
  📊 Total in range: 4.0 h | Epic in range: 0.0 h
  🔍 Epic tasks out of range: 7
    - IT-20942 (8.0 h) due 2025-10-17
    - IT-22564 (8.0 h) due 2025-11-28
    - IT-22566 (12.0 h) due 2025-12-19
    - IT-20578 (44.0 h) due 2026-02-20
    - IT-20577 (64.0 h) due 2026-02-20
    - IT-11523 (8.0 h) due 2026-02-20
    - IT-11789 (2.0 h) due 2100-01-01
 modificando IT-20942 due 2100-01-01
✔️ Ratio tras forzar IT-20942: 8.0 / (50.0) = 16.0%
 modificando IT-22564 due 2100-01-01
✔️ Ratio tras forzar IT-22

In [6]:
with open("planificacion.json", "w") as f:
    json.dump(scheduled, f, default=str, indent=2)